Script de update datos MongoDB en cluster multidomain

In [1]:
import pandas as pd
import numpy as np
import os
import json
import random
import pymongo
from pymongo import MongoClient
import time
from pprint import pprint
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
import matplotlib.pyplot as plt
import psutil
import uuid
from bson import objectid

In [2]:
#Los resultados de medidas de tiempo en carga por dominios se almacenan en estos objetos.
#Se itera durante 100 iteraciones para sacar medias
#repeticiones
repeats = 100

In [3]:
#Ficheros de salida
resultados_etl_update = '../Results/MongoDB/MongoDBUpdate_test_{}.csv'

In [4]:
def save_results_to_csv(results,file):
    #Guardamos los resultados en csv
    from datetime import datetime
    
    csv_df = pd.DataFrame(results, columns=['Registros', 'Tiempo', 'CPU','Memoria'])
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S")
    csv_df.to_csv(file.format(str(dia)))

In [5]:
#Conexion unica a colecciones
connection = MongoClient('localhost', 27017, w=3) #Conexion con WriteConcern a 3 (primario y dos nodos consolidados)

#Creamos una base de datos para el trabajo llamada TFM
tfm_mongo_db = connection["tfm_mongo_database"]
tfm_mongo_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, w=3), 'tfm_mongo_database')

In [6]:
CustomerProfileCollection = tfm_mongo_db["CustomerProfileCollection"]
CurrentAccountCollection = tfm_mongo_db["CurrentAccountCollection"]
PositionKeepingCollection = tfm_mongo_db["PositionKeepingCollection"]
print(connection.list_database_names())

['admin', 'config', 'local', 'tfm_mongo_database']


# Update test multidomain

In [40]:
pipeline = [
        {
          '$match': {
              'Status': 'Enabled'
          }  
        }, 
        {
            '$lookup': {
                'from' : 'PositionKeepingCollection',
                'localField' : 'AccountId',
                'foreignField' : 'AccountId',
                'as' : 'PositionKeeping'
            }
        },
        {
            '$unwind': '$PositionKeeping'
        },
        { 
            '$limit': 100
        },
        {
          '$match': {
              'AccountInfo.SchemeName': { '$regex': 'UK.*' },              
              'PositionKeeping.Amount.Currency': { '$regex': ".*USA.*" }
          }  
        },
        {
            '$project': {
                '_id':0,
                'AccountId': 1            
            }
        }
]

In [41]:
time_inicial = 0
time_final = 0
registers = []
lista = []
for iteracion in range(0,repeats): 
   
    time_inicial = time.time()
    result = CurrentAccountCollection.aggregate(pipeline)

    for doc in result:
        #Update
        #print(doc['AccountId'])
        lista.append(doc['AccountId'])
        CurrentAccountCollection.update_one({'AccountId':doc['AccountId']},{'$set': {'Status':'Disabled_ch'}})
        
    time_final = time.time() 
    used_cpu = psutil.cpu_percent()
    mem_used = psutil.virtual_memory().percent
    # Tupla con numero de registros, tiempo parcial de la transacción y tiempo acumulado de trxs
    total_time = round(time_final - time_inicial,3)
    registers.append((iteracion + 1, total_time ,used_cpu, mem_used))

In [42]:
print(CurrentAccountCollection.count_documents({'Status': 'Disabled_ch'}))

1


In [43]:
registers

[(1, 0.442, 2.6, 72.5),
 (2, 0.442, 16.7, 72.5),
 (3, 0.446, 24.7, 72.6),
 (4, 0.438, 18.5, 72.6),
 (5, 0.442, 22.8, 72.6),
 (6, 0.446, 22.2, 72.6),
 (7, 0.433, 17.0, 72.6),
 (8, 0.438, 18.9, 72.6),
 (9, 0.435, 16.7, 72.6),
 (10, 0.445, 19.3, 72.6),
 (11, 0.436, 17.2, 72.6),
 (12, 0.434, 16.9, 72.6),
 (13, 0.44, 18.2, 72.6),
 (14, 0.434, 16.5, 72.5),
 (15, 0.449, 19.0, 72.5),
 (16, 0.44, 18.6, 72.6),
 (17, 0.434, 16.6, 72.6),
 (18, 0.434, 17.9, 72.6),
 (19, 0.435, 18.4, 72.6),
 (20, 0.436, 16.6, 72.6),
 (21, 0.435, 17.9, 72.6),
 (22, 0.436, 17.6, 72.6),
 (23, 0.441, 16.7, 72.6),
 (24, 0.436, 17.2, 72.6),
 (25, 0.434, 17.2, 72.6),
 (26, 0.437, 17.0, 72.6),
 (27, 0.435, 18.8, 72.6),
 (28, 0.434, 17.7, 72.6),
 (29, 0.448, 17.7, 72.6),
 (30, 0.443, 18.9, 72.6),
 (31, 0.442, 18.5, 72.6),
 (32, 0.448, 17.7, 72.6),
 (33, 0.446, 18.4, 72.6),
 (34, 0.441, 21.9, 72.6),
 (35, 0.445, 17.3, 72.6),
 (36, 0.437, 18.3, 72.6),
 (37, 0.439, 18.9, 72.5),
 (38, 0.435, 17.6, 72.5),
 (39, 0.444, 20.7, 72.5)

In [44]:
#Guardamos los resultados Customer Profile
save_results_to_csv(registers,resultados_etl_update)

In [45]:
connection.close()
print('Conexion cerrada')

Conexion cerrada
